In [1]:
from portiloop_software.portiloop_python.ANN.data.mass_data import SingleSubjectDataset, read_pretraining_dataset, read_spindle_trains_labels
from portiloop_software.portiloop_python.ANN.utils import get_configs

/project/portiloop-training/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import numpy as np

file1 = '/project/portiloop-training/portiloop_software/dataset/6_segListSrcDataLoc_p1.txt'
file2 = '/project/portiloop-training/portiloop_software/dataset/7_segListSrcDataLoc_p2.txt'

In [3]:
data1 = np.genfromtxt(file1, delimiter='\t', dtype=str)[1:, [1, 4]]
data2 = np.genfromtxt(file2, delimiter='\t', dtype=str)[1:, [1, 4]]

In [4]:
all_segments = np.concatenate((data1, data2), axis=0)
all_segments.shape

(749, 2)

In [5]:
import numpy as np

filename_lacourse = '/project/portiloop-training/portiloop_software/dataset/detectionVector_01-01-0001.csv'
# Load file
spindle_labels = np.loadtxt(filename_lacourse, delimiter=',')

This line gets the starting index in seconds for each segment for each subject and puts it in a dictionary where the key is the subject id and the values are lists of the starting indexes. Each segment will hen be exactly 115 seconds.

In [6]:
segments_dict = {subject: (all_segments[np.where(all_segments[:, 0] == subject)[0], 1].astype(float) * 250).astype(int).tolist() for subject in np.unique(all_segments[:, 0])}

In [7]:
segments_dict

{'01-01-0001': [2054733,
  2122233,
  3637233,
  4582233,
  4849733,
  4982233,
  6237233,
  6269733,
  6339733,
  6382233],
 '01-01-0002': [745469, 2502969, 5560469],
 '01-01-0003': [3800242, 5047742, 5782742],
 '01-01-0004': [1261480, 3051480, 3611480],
 '01-01-0005': [787477, 1163477, 1303477],
 '01-01-0006': [3782239, 4772239, 4872239],
 '01-01-0007': [1352880,
  2070380,
  4005150,
  4207880,
  5332880,
  5455380,
  6300380,
  6445380,
  6522880,
  6605380],
 '01-01-0008': [2689748,
  3147248,
  4424748,
  4469748,
  4507248,
  4579748,
  5629748,
  5749248,
  5791748,
  5851748],
 '01-01-0010': [870471,
  2257971,
  3355471,
  4362971,
  5027971,
  5482971,
  5946971,
  5989471,
  6036971,
  6091971],
 '01-01-0011': [868210,
  1840710,
  3570710,
  3870710,
  4045710,
  4445710,
  4814710,
  5483210,
  5630710,
  5710710],
 '01-01-0012': [2460241,
  2502741,
  3457241,
  4134741,
  4383241,
  4810241,
  5109741,
  5284741,
  5344741,
  7618741],
 '01-01-0013': [512745,
  1110245,

Load the Wamsley data

In [47]:
experiment_name = 'MASS'
seed = 42

config = get_configs(experiment_name, False, seed)

data = read_pretraining_dataset(config['MASS_dir'])
labels = read_spindle_trains_labels(config['old_dataset'])

In [50]:
from portiloop_software.portiloop_python.ANN.data.mass_data import read_sleep_staging_labels

ss_labels = read_sleep_staging_labels(config['path_dataset'])


In [54]:
wamsley_labels = {}
all_keys = list(data.keys())
for subject in all_keys:
    try:
        dataset = SingleSubjectDataset(subject, data=data, labels=labels, config=config, ss_labels=ss_labels)
        wamsley_labels[subject] = dataset.full_labels
    except KeyError:
        continue

In [57]:
for wamsley_label in wamsley_labels.keys():
    if wamsley_label not in segments_dict.keys():
        continue

    

01-01-0034
01-01-0039
01-02-0007
01-02-0008
01-02-0009
01-02-0016
01-02-0018
01-03-0055
01-05-0021
01-05-0026


Load the MODA Data

In [8]:
subjects_path = '/project/portiloop-training/portiloop_software/dataset/subject_sequence_full_big.txt'
data_path = '/project/portiloop-training/portiloop_software/dataset/dataset_classification_full_big_250_matlab_standardized_envelope_pf.txt'

In [9]:
import pandas as pd

data = pd.read_csv(data_path, header=None).to_numpy()


In [10]:
subjects = np.genfromtxt(subjects_path, dtype=str)

In [11]:
mask = (data[:, 3] != -1) & (data[:, 3] != -2)
data_filtered = data[mask]

In [12]:
len(data_filtered) / 115 / 250

695.0

In [13]:
data_filtered

array([[-0.1674333 ,  0.3998575 ,  0.5673624 ,  0.        ],
       [-0.104413  ,  0.3958714 ,  0.5783259 ,  0.        ],
       [-0.0406625 ,  0.3920332 ,  0.5835297 ,  0.        ],
       ...,
       [-0.1219364 ,  0.2911943 ,  0.6499147 ,  0.        ],
       [-0.1322542 ,  0.2894326 ,  0.6500487 ,  0.        ],
       [-0.09577711,  0.2870451 ,  0.6502095 ,  0.        ]])

In [14]:
split_data = np.array(np.split(data_filtered[:, -1], int(len(data_filtered) / (115 * 250))))
split_data.shape

(695, 28750)

In [17]:
subjects.shape

(162, 3)

In [18]:
moda_index_dict = {}
for subject in subjects[:, 0]:
    s = subjects[subjects[:, 0] == subject][:, 1:].astype(int)[0]
    moda_index_dict[subject] = list(range(s[0], s[1]))

In [19]:
moda_labels = {}

for subject in moda_index_dict.keys():
    moda_labels[subject] = np.hstack(split_data[moda_index_dict[subject]])

In [20]:
moda_labels['01-04-0018'].shape

(86250,)

Putting everything together:

In [23]:
len(moda_labels['01-01-0001'])

287500

In [25]:
len(spindle_labels)

7638500

In [29]:
final_lacourse_labels = []

subject_id = '01-01-0001'

for sequence_index in segments_dict[subject_id]:
    final_lacourse_labels += spindle_labels[sequence_index:sequence_index + 115 * 250].tolist()

# for subject in moda_labels.keys():
#     if subject not in wamsley_labels.keys():
#         continue

#     # Get the necessary indexes of the wamsley labels
#     for 

In [31]:
len(final_lacourse_labels)

287500

In [36]:
from sklearn.metrics import precision_recall_fscore_support

In [40]:
sum(np.array(final_lacourse_labels) == moda_labels['01-01-0001']) / len(final_lacourse_labels)

0.9432069565217391

In [41]:
def get_metrics(predictions, labels):
    """
    Compute the F1, precision and recall for spindles from a true positive count, false positive count and false negative count
    """
    # n_classes = len(torch.unique(labels))

    # tp = torch.zeros(n_classes)
    # tn = torch.zeros(n_classes)
    # fp = torch.zeros(n_classes)
    # fn = torch.zeros(n_classes)

    # for i in range(n_classes):
    #     tp[i] = torch.sum((predictions == i) & (labels == i))
    #     tn[i] = torch.sum((predictions != i) & (labels != i))
    #     fp[i] = torch.sum((predictions == i) & (labels != i))
    #     fn[i] = torch.sum((predictions != i) & (labels == i))

    # epsilon = 1e-7

    # accuracy = torch.sum(tp) / torch.sum(tp + tn + fp + fn)
    # recall = torch.mean(tp / (tp + fn + epsilon))
    # precision = torch.mean(tp / (tp + fp + epsilon))
    # f1_score = torch.mean(2 * precision * recall / (precision + recall + epsilon))

    # return accuracy, recall, precision, f1_score
    acc = (predictions == labels).float().mean()
    predictions = predictions.float()
    labels = labels.float()

    # Get the true positives, true negatives, false positives and false negatives
    tp = (labels * predictions)
    tn = ((1 - labels) * (1 - predictions))
    fp = ((1 - labels) * predictions)
    fn = (labels * (1 - predictions))

    tp_sum = tp.sum().to(torch.float32).item()
    fp_sum = fp.sum().to(torch.float32).item()
    fn_sum = fn.sum().to(torch.float32).item()
    epsilon = 1e-7

    precision = tp_sum / (tp_sum + fp_sum + epsilon)
    recall = tp_sum / (tp_sum + fn_sum + epsilon)

    f1 = 2 * (precision * recall) / (precision + recall + epsilon)

    return acc, f1, precision, recall


In [39]:
precision_recall_fscore_support(np.array(final_lacourse_labels), moda_labels['01-01-0001'], average='binary')

(0.7450109194969501, 0.674323495330925, 0.707906976744186, None)

In [42]:
import torch

get_metrics(torch.tensor(final_lacourse_labels), torch.from_numpy(moda_labels['01-01-0001']))

(tensor(0.9432), 0.7079069268656748, 0.6743234953286268, 0.7450109194941448)